# Tensorflow Model

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from time import localtime
import matplotlib.pyplot as plt
import ipdb

In [2]:
data_dir = '../NCAA/'
data_file = data_dir + 'MarchMadnessFeatures.csv'
df_features = pd.read_csv(data_file)

X = df_features.iloc[:,1:]
xDim = np.shape(X)[1]
X = X.values.reshape(-1,xDim)
Y = df_features.Result.values
Y = Y.reshape(-1,1)

# Y = np.array([Y, -(Y-1)]).T  # The model currently needs one column for each class

from sklearn.model_selection import train_test_split
X, X_test, Y, Y_test = train_test_split(X, Y, test_size=0.20, stratify=Y)

print('Feature vector dimension is: %.2f' % xDim)

Feature vector dimension is: 34.00


In [7]:
# Parameter
learning_rate = 0.0003
training_epochs = 150
batch_size = 50 #25
display_step = 5


# Network Parameters
n_hidden_1 = 200 
n_hidden_2 = 200
n_input = 34 # Number of feature
n_classes = 2 


sess = tf.InteractiveSession()

# Input placeholders
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, n_input], name='x-input')
    y_ = tf.placeholder(tf.int64, [None,1], name='y-input')

# We can't initialize these variables to 0 - the network will get stuck.
def weight_variable(shape):
    """Create a weight variable with appropriate initialization."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    """Create a bias variable with appropriate initialization."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

def nn_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.sigmoid):
    """Reusable code for making a simple neural net layer.
    
    It does a matrix multiply, bias add, and then uses relu to nonlinearize.
    It also sets up name scoping so that the resultant graph is easy to read,
    and adds a number of summary ops.
    """
        # Adding a name scope ensures logical grouping of the layers in the graph.
    with tf.name_scope(layer_name):
        # This Variable will hold the state of the weights for the layer
        with tf.name_scope('weights'):
            weights = weight_variable([input_dim, output_dim])
            variable_summaries(weights)
        with tf.name_scope('biases'):
            biases = bias_variable([output_dim])
            variable_summaries(biases)
        with tf.name_scope('Wx_plus_b'):
            preactivate = tf.matmul(input_tensor, weights) + biases
            tf.summary.histogram('pre_activations', preactivate)
        activations = act(preactivate, name='activation')
        tf.summary.histogram('activations', activations)
        return activations
  
    
hidden1 = nn_layer(x, n_input, n_hidden_1, 'layer1')
# hidden2 = nn_layer(dropped, n_input, n_hidden_2, 'layer2')

with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    tf.summary.scalar('dropout_keep_probability', keep_prob)
    dropped = tf.nn.dropout(hidden1, keep_prob)

    
# Do not apply softmax activation yet, see below.
y = nn_layer(dropped, n_hidden_1, 1, 'layerout', act=tf.identity)

with tf.name_scope('cross_entropy'):
    # The raw formulation of cross-entropy,
    #
    # tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.softmax(y)),
    #                               reduction_indices=[1]))
    #
    # can be numerically unstable.
    #
    # So here we use tf.losses.sparse_softmax_cross_entropy on the
    # raw logit outputs of the nn_layer above.
    with tf.name_scope('total'):
        cross_entropy = tf.losses.sigmoid_cross_entropy(labels=y_, predictions=y)
tf.summary.scalar('cross_entropy', cross_entropy)

with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar('accuracy', accuracy)

# Merge all the summaries and write them out to /tmp/mnist_logs (by default)
merged = tf.summary.merge_all()
summaries_dir = data_dir+'logs'
train_writer = tf.summary.FileWriter(summaries_dir + '/train',
                                      sess.graph)
test_writer = tf.summary.FileWriter(summaries_dir + '/test')
init = tf.global_variables_initializer()



TypeError: sigmoid_cross_entropy() got an unexpected keyword argument 'labels'

In [ ]:
# Training cycle
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(len(X)/batch_size)
        X_batches = np.array_split(X, total_batch)
        Y_batches = np.array_split(Y, total_batch)
        # Train for all bactches over all batches
        for i in range(total_batch):
#             ipdb.set_trace()
            batch_x, batch_y = X_batches[i], Y_batches[i]
            summary, _ = sess.run([merged, train_step], feed_dict={x: batch_x,
                                                                   y_: batch_y,
                                                                   keep_prob: 0.8})
        train_writer.add_summary(summary, epoch)
        # Record test set summaries and test accuracy
        if epoch % display_step == 0:
            summary, acc = sess.run([merged, accuracy], feed_dict={x: X_test,
                                                                   y_: Y_test,
                                                                   keep_prob: 1.0})
            test_writer.add_summary(summary, epoch)
            print('Accuracy at step %s: %s' % (summary, epoch))
    print("Training Complete")
    
train_writer.close()
test_writer.close()

In [6]:
# Test model
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
# Calculate accuracy
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
print("Accuracy:", accuracy.eval({x: X_test, y: Y_test}))
global result 
result = tf.argmax(pred, 1).eval({x: X_test, y: Y_test})

NameError: name 'pred' is not defined

In [ ]:
tf.reset_default_graph